In [1]:
import os
import subprocess

def git_repo_root():
    # Run the 'git rev-parse --show-toplevel' command to get the root directory of the Git repository
    try:
        root = subprocess.check_output(['git', 'rev-parse', '--show-toplevel'], universal_newlines=True).strip()
        return root
    except subprocess.CalledProcessError:
        # Handle the case where the current directory is not inside a Git repository
        return None

# Get the root directory of the Git repository
git_root = git_repo_root()

if git_root:
    # Change the working directory to the root of the Git repository
    os.chdir(git_root)
    print(f"Changed working directory to: {git_root}")
else:
    print("Not inside a Git repository.")

Changed working directory to: /Users/stephenrobbins/Documents/Documents-Stephens_MacBook_Pro/Python/Random/Diffusion_project


In [2]:

import torch
import torchvision
from matplotlib import pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from diffusion import BridgeDiffusionVPSDE
from training import train_score_network_mnist
from denoising_diffusion_pytorch import Unet


/Users/stephenrobbins/opt/anaconda3/envs/bridge_test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-08 16:04:25.258254: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
device = 'cpu'

''' hyper params'''
n_epochs =   50
batch_size =  32

In [5]:
class CorrectEMNISTOrientation(object):
    def __call__(self, img):
        # Rotate 90 degrees counter-clockwise
        img = transforms.functional.rotate(img, -90)
        # Flip horizontally
        img = transforms.functional.hflip(img)
        return img

tfm = transforms.Compose([transforms.ToTensor(), transforms.Pad(2)])
emnist_tfm = transforms.Compose([transforms.ToTensor(),transforms.Pad(2),CorrectEMNISTOrientation()])

# Load MNIST dataset
mnist_data = torchvision.datasets.MNIST('./data/', transform=tfm, download=True)
mnist_loader = DataLoader(mnist_data, batch_size=batch_size, shuffle=True)

# Load EMNIST dataset
emnist_data = torchvision.datasets.EMNIST('./data/', 'letters', transform=emnist_tfm, download=True)
emnist_loader = DataLoader(emnist_data, batch_size=batch_size, shuffle=True)

def data_y(n):
    # Create a new DataLoader with batch_size = n
    temp_loader = DataLoader(emnist_data, batch_size=n, shuffle=True)

    # Fetch one batch of data
    for data, labels in temp_loader:
        return data # This returns n samples

model = Unet(out_dim=1, channels = 2, dim = 32).to(device)
# I think the noise level might affect training. I traned 50 epochs at 
# .2 noise and didn't get good results. Note we want noise to be symmetric 
# so thats why bmin=bmax right now
sde = BridgeDiffusionVPSDE(data_y,  bmin=0.5, bmax=0.5, device = device)

In [ ]:
train_score_network_mnist(mnist_loader, model, sde, epochs=50, bridge=True)
torch.save(model.state_dict(), 'model_weights.pth')

In [ ]:
model = Unet(out_dim=1, channels = 2, dim = 32).to(device)  
model.load_state_dict(torch.load('model_weights.pth'))
sde.plot_diffusion(model, data_shape=(5, 1, 32, 32))